##### 1. Use the `random_circuit` function from `qbraid.interfaces` to generate random circuits across three frameworks

##### 2. Use the qBraid transpiler to convert a Bell circuit in one library to another. For example, Qiskit to Cirq.

##### 3. Use code snippets to create a simple notebook and share that notebook with another user in the tutorial

##### 4. Create a custom environment and then share that environment with another user in the tutorial

##### 5. The code below defines a simulation of VQE for the 2D Ising model. It shows how to run the simulation using CPU only and defines a GPU simulator using qsim. Run the simulation using the CPU backend and compare the time to that of the GPU backend. Note that you should stick with using the default qBraid Lab environment for this demo.

In [ ]:
import cirq, qsimcirq, time, random
import numpy as np

def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length - 1, length)
    # links within a column
    jc = rand2d(length, length - 1)
    return (h, jr, jc)

h, jr, jc = random_instance(3)

def prepare_plus_layer(length):
    for i in range(length):
        for j in range(length):
            yield cirq.H(cirq.GridQubit(i, j))

def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""

    # Define the gate once and then re-use it for each Operation.
    rot = cirq.XPowGate(exponent=half_turns)

    # Create an X rotation Operation for each qubit in the grid.
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    cz_gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i + 1, j)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i, j + 1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

length = 5
qubits = cirq.GridQubit.square(length)
h, jr, jc = random_instance(length)

circuit = cirq.Circuit()
circuit.append(prepare_plus_layer(length))
circuit.append(rot_z_layer(h,0.3))
circuit.append(rot_11_layer(jr,jc,0.3))
circuit.append(rot_x_layer(length, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))

In [ ]:
# Default CPU simulator
simulator = cirq.Simulator()

# cuQuantum GPU simulator
ngpus = 1
qsim_options = qsimcirq.QSimOptions(
    max_fused_gate_size = 2,
    use_gpu = True,
    gpu_mode = 1 # gpu_mode: use CUDA if set to 0 (default value) or use the NVIDIA cuStateVec library if set to any other value.
)

simulator_cuQ=qsimcirq.QSimSimulator(qsim_options)

In [ ]:
# Default CPU simulator VQE

start = time.time()
results = simulator.run(circuit, repetitions=10000)
# print(results.histogram(key='x'))
cpu_time = time.time() - start
print("CPU runtime: {:.24}s".format(cpu_time))
# print(f'Simulated ground-state energy expectation value is {obj_func(results)}')

##### 6. Everyone is talking about QuEra's Aquila machine, but many may not have actually run a program on it yet. Use the code snippets feature of qBraid Lab to create and submit an Analog Hamiltonian Simulation to Aquila.

##### 7. See if you can run a circuit simulation using qiskit aer gpu using the default environment on qBraid Lab.